## 1. Configuración e Importación de Librerías

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")

✅ Librerías cargadas correctamente


## 2. Carga de Datos

In [2]:
# Rutas de los archivos
PATH_SEGMENTACION = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\data\segmentacion_final\activos_segmentados_kmeans.csv'
PATH_PRECIOS = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\data\prices_train.csv'

# Cargar datos de segmentación
df_segmentacion = pd.read_csv(PATH_SEGMENTACION)

# Cargar datos de precios
df_precios = pd.read_csv(PATH_PRECIOS, parse_dates=['date'], index_col='date')

# Excluir SPY de la estrategia
df_segmentacion = df_segmentacion[df_segmentacion['ticker'] != 'SPY'].copy()

print(f"📊 Activos en segmentación: {len(df_segmentacion)}")
print(f"📈 Activos en precios: {len(df_precios.columns)}")
print(f"📅 Período de precios: {df_precios.index.min().strftime('%Y-%m-%d')} a {df_precios.index.max().strftime('%Y-%m-%d')}")
print(f"\n🔍 Distribución por segmento:")
print(df_segmentacion.groupby(['segmento', 'segmento_nombre']).size().reset_index(name='count'))

📊 Activos en segmentación: 467
📈 Activos en precios: 468
📅 Período de precios: 2021-01-04 a 2023-12-29

🔍 Distribución por segmento:
   segmento          segmento_nombre  count
0        -1  Outliers_Riesgo_Extremo     29
1         0              Conservador     50
2         1         Alto_Rendimiento     96
3         2                 Moderado     94
4         3                  Estable    198


## 3. Definición de Perfiles de Inversión y Criterio de Selección

In [3]:
# Definición de perfiles de inversión con distribución de activos (total = 10)
# CORRECCIÓN: Ajustado para mejor exposición a crecimiento en mercados alcistas
PERFILES_INVERSION = {
    'Conservador': {
        'descripcion': 'Prioriza la estabilidad y preservación del capital',
        'distribucion': {3: 6, 0: 2, 2: 2},  # 60% Estable, 20% Conservador, 20% Moderado
        'clusters_nombres': ['Estable (C3)', 'Conservador (C0)', 'Moderado (C2)']
    },
    'Moderado': {
        'descripcion': 'Balance entre crecimiento y estabilidad',
        'distribucion': {1: 4, 2: 3, 3: 3},  # 40% Alto Rendimiento, 30% Moderado, 30% Estable
        'clusters_nombres': ['Alto Rendimiento (C1)', 'Moderado (C2)', 'Estable (C3)']
    },
    'Agresivo': {
        'descripcion': 'Busca alto rendimiento con exposición a beta alto (solo outliers positivos)',
        'distribucion': {1: 7, 2: 2, -1: 1},  # 70% Alto Rendimiento, 20% Moderado, 10% Outliers+
        'clusters_nombres': ['Alto Rendimiento (C1)', 'Moderado (C2)', 'Outliers+ (C-1)']
    },
    'Especulativo': {
        'descripcion': 'Máximo rendimiento potencial con outliers positivos de alto potencial',
        'distribucion': {1: 5, -1: 3, 2: 2},  # 50% Alto Rendimiento, 30% Outliers+, 20% Moderado
        'clusters_nombres': ['Alto Rendimiento (C1)', 'Outliers+ (C-1)', 'Moderado (C2)']
    },
    'Normal': {
        'descripcion': 'Exposición balanceada a TODOS los clusters (2 de cada uno, solo outliers positivos)',
        'distribucion': {-1: 2, 0: 2, 1: 2, 2: 2, 3: 2},  # 2 de cada cluster = 10 activos
        'clusters_nombres': ['Outliers+ (C-1)', 'Conservador (C0)', 'Alto Rendimiento (C1)', 'Moderado (C2)', 'Estable (C3)']
    }
}

def calcular_momentum_6m(df_precios, tickers):
    """
    Calcula el momentum de 6 meses para cada ticker.
    Momentum = (Precio actual / Precio hace 6 meses) - 1
    """
    momentum_dict = {}
    dias_6_meses = 126  # ~6 meses de días de trading
    
    for ticker in tickers:
        if ticker in df_precios.columns:
            precios = df_precios[ticker].dropna()
            if len(precios) >= dias_6_meses:
                precio_actual = precios.iloc[-1]
                precio_6m_atras = precios.iloc[-dias_6_meses]
                momentum_dict[ticker] = (precio_actual / precio_6m_atras) - 1
            else:
                # Si no hay suficientes datos, usar todo el período disponible
                precio_actual = precios.iloc[-1]
                precio_inicio = precios.iloc[0]
                momentum_dict[ticker] = (precio_actual / precio_inicio) - 1
        else:
            momentum_dict[ticker] = 0  # Valor por defecto si no hay datos
    
    return momentum_dict

def calcular_score_momentum(df, df_precios):
    """
    Calcula un score orientado a crecimiento con momentum.
    Inspirado en Peter Lynch y William O'Neil (CANSLIM).
    
    CORRECCIÓN: En mercados alcistas, beta alto es BUENO (amplifica ganancias)
    
    Score_Momentum = 0.35 × Return_norm + 
                     0.30 × Momentum_6m_norm + 
                     0.15 × Sharpe_norm + 
                     0.20 × Beta_norm  (CORREGIDO: ahora beta alto es mejor)
    
    - Retorno Anualizado (35%): Prioriza rendimiento absoluto
    - Momentum 6 meses (30%): Captura tendencias recientes (aumentado)
    - Sharpe Ratio (15%): Control mínimo de eficiencia riesgo-retorno
    - Beta (20%): En mercados alcistas, beta > 1 amplifica ganancias
    """
    df = df.copy()
    
    # Normalizar métricas usando min-max scaling
    def normalize(series):
        min_val = series.min()
        max_val = series.max()
        if max_val - min_val == 0:
            return pd.Series([0.5] * len(series), index=series.index)
        return (series - min_val) / (max_val - min_val)
    
    # Calcular momentum de 6 meses
    tickers = df['ticker'].tolist()
    momentum_dict = calcular_momentum_6m(df_precios, tickers)
    df['momentum_6m'] = df['ticker'].map(momentum_dict)
    
    # Normalizar cada métrica
    df['return_norm'] = normalize(df['return_annualized'])
    df['momentum_norm'] = normalize(df['momentum_6m'])
    df['sharpe_norm'] = normalize(df['sharpe_ratio'])
    # CORRECCIÓN: Beta alto es BUENO en mercados alcistas (amplifica ganancias)
    # Normalizamos beta directamente (mayor beta = mayor score)
    df['beta_norm'] = normalize(df['beta'])
    
    # Calcular score compuesto orientado a momentum y crecimiento
    df['score_compuesto'] = (
        0.35 * df['return_norm'] +      # 35% peso en retorno
        0.30 * df['momentum_norm'] +    # 30% peso en momentum (aumentado)
        0.15 * df['sharpe_norm'] +      # 15% peso en Sharpe (reducido)
        0.20 * df['beta_norm']          # 20% peso en beta (CORREGIDO: beta alto es mejor)
    )
    
    return df

# Calcular score para todos los activos (usando fórmula de Momentum CORREGIDA)
df_segmentacion = calcular_score_momentum(df_segmentacion, df_precios)

print("✅ Score de Momentum calculado para todos los activos")
print("\n📊 FÓRMULA UTILIZADA (Método Momentum CORREGIDO - orientado a crecimiento):")
print("   Score = 0.35×Return + 0.30×Momentum_6m + 0.15×Sharpe + 0.20×Beta")
print("\n⚠️ CORRECCIÓN APLICADA: Beta alto ahora es POSITIVO (amplifica ganancias en mercados alcistas)")
print("\n🏆 Top 5 activos por score de momentum:")
display(df_segmentacion.nlargest(5, 'score_compuesto')[['ticker', 'segmento_nombre', 'return_annualized', 'momentum_6m', 'sharpe_ratio', 'beta', 'score_compuesto']])

✅ Score de Momentum calculado para todos los activos

📊 FÓRMULA UTILIZADA (Método Momentum CORREGIDO - orientado a crecimiento):
   Score = 0.35×Return + 0.30×Momentum_6m + 0.15×Sharpe + 0.20×Beta

⚠️ CORRECCIÓN APLICADA: Beta alto ahora es POSITIVO (amplifica ganancias en mercados alcistas)

🏆 Top 5 activos por score de momentum:


,ticker,segmento_nombre,return_annualized,momentum_6m,sharpe_ratio,beta,score_compuesto
308,NVDA,Outliers_Riesgo_Extremo,0.561655,0.167649,1.007627,2.130628,0.718758
288,MPC,Outliers_Riesgo_Extremo,0.542491,0.262209,1.326416,0.783717,0.674179
258,LLY,Outliers_Riesgo_Extremo,0.525457,0.263318,1.415592,0.516283,0.662032
307,NUE,Alto_Rendimiento,0.494666,0.050903,1.066407,1.185375,0.660712
339,PWR,Alto_Rendimiento,0.471237,0.102652,1.177923,1.081092,0.656168


## 4. Función de Selección de Portafolio

In [4]:
def seleccionar_portafolio(perfil, df_seg, seed=42):
    """
    Selecciona los 10 mejores activos para un perfil de inversión dado.
    
    Args:
        perfil: Nombre del perfil de inversión
        df_seg: DataFrame con la segmentación y scores
        seed: Semilla para reproducibilidad en caso de empates
    
    Returns:
        DataFrame con los activos seleccionados
    
    IMPORTANTE: Para el cluster de outliers (-1), solo se seleccionan
    activos con rendimiento positivo para evitar outliers "malos".
    """
    np.random.seed(seed)
    
    config = PERFILES_INVERSION[perfil]
    distribucion = config['distribucion']
    
    activos_seleccionados = []
    
    for cluster, n_activos in distribucion.items():
        # Filtrar activos del cluster
        df_cluster = df_seg[df_seg['segmento'] == cluster].copy()
        
        # FILTRO ESPECIAL: Para outliers (cluster -1), solo incluir los de rendimiento POSITIVO
        if cluster == -1:
            df_cluster = df_cluster[df_cluster['return_annualized'] > 0]
            if len(df_cluster) == 0:
                print(f"⚠️ No hay outliers con rendimiento positivo disponibles")
                continue
        
        if len(df_cluster) == 0:
            print(f"⚠️ No hay activos disponibles en el cluster {cluster}")
            continue
        
        # Ordenar por score compuesto y seleccionar los mejores
        df_cluster = df_cluster.sort_values('score_compuesto', ascending=False)
        
        # Seleccionar los n mejores activos
        n_disponibles = min(n_activos, len(df_cluster))
        seleccionados = df_cluster.head(n_disponibles)
        
        activos_seleccionados.append(seleccionados)
    
    # Combinar todos los activos seleccionados
    df_portafolio = pd.concat(activos_seleccionados, ignore_index=True)
    
    # Agregar columna de peso (equi-ponderado)
    df_portafolio['peso'] = 1 / len(df_portafolio)
    
    return df_portafolio

print("✅ Función de selección de portafolio definida")
print("⚠️ NOTA: Los outliers (cluster -1) solo incluyen activos con rendimiento POSITIVO")

✅ Función de selección de portafolio definida
⚠️ NOTA: Los outliers (cluster -1) solo incluyen activos con rendimiento POSITIVO


## 5. Funciones de Visualización

In [5]:
def graficar_precios_normalizados(tickers, df_precios, titulo="Evolución de Precios Normalizados (Base 100)"):
    """
    Crea un gráfico de precios normalizados para los activos seleccionados.
    """
    # Filtrar solo los tickers que existen en el DataFrame de precios
    tickers_disponibles = [t for t in tickers if t in df_precios.columns]
    
    if len(tickers_disponibles) == 0:
        print("⚠️ No hay tickers disponibles en los datos de precios")
        return None
    
    # Obtener precios de los tickers seleccionados
    df_subset = df_precios[tickers_disponibles].copy()
    
    # Normalizar a base 100
    df_normalizado = (df_subset / df_subset.iloc[0]) * 100
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Colores para cada activo
    colors = px.colors.qualitative.Set3[:len(tickers_disponibles)]
    
    for i, ticker in enumerate(tickers_disponibles):
        fig.add_trace(go.Scatter(
            x=df_normalizado.index,
            y=df_normalizado[ticker],
            mode='lines',
            name=ticker,
            line=dict(width=2, color=colors[i % len(colors)]),
            hovertemplate=f'{ticker}<br>Fecha: %{{x}}<br>Valor: %{{y:.2f}}<extra></extra>'
        ))
    
    # Agregar línea de referencia en 100
    fig.add_hline(y=100, line_dash="dash", line_color="gray", opacity=0.5,
                  annotation_text="Base 100", annotation_position="right")
    
    fig.update_layout(
        title=dict(text=titulo, font=dict(size=16)),
        xaxis_title="Fecha",
        yaxis_title="Valor Normalizado (Base 100)",
        template="plotly_white",
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        ),
        height=600,
        margin=dict(b=100)
    )
    
    return fig

def mostrar_resumen_portafolio(df_portafolio, perfil):
    """
    Muestra un resumen formateado del portafolio seleccionado.
    """
    config = PERFILES_INVERSION[perfil]
    
    print(f"\n{'='*70}")
    print(f"📊 PORTAFOLIO SELECCIONADO - PERFIL: {perfil.upper()}")
    print(f"{'='*70}")
    print(f"📝 Descripción: {config['descripcion']}")
    print(f"📈 Clusters: {', '.join(config['clusters_nombres'])}")
    print(f"\n{'─'*70}")
    
    # Tabla de activos (incluye momentum y beta para el nuevo score)
    display_cols = ['ticker', 'segmento_nombre', 'return_annualized', 'momentum_6m', 
                    'sharpe_ratio', 'beta', 'score_compuesto', 'peso']
    
    df_display = df_portafolio[display_cols].copy()
    df_display.columns = ['Ticker', 'Segmento', 'Retorno Anual', 'Momentum 6m', 
                          'Sharpe', 'Beta', 'Score', 'Peso']
    
    # Formatear porcentajes
    df_display['Retorno Anual'] = df_display['Retorno Anual'].apply(lambda x: f"{x*100:.2f}%")
    df_display['Momentum 6m'] = df_display['Momentum 6m'].apply(lambda x: f"{x*100:.2f}%")
    df_display['Peso'] = df_display['Peso'].apply(lambda x: f"{x*100:.1f}%")
    df_display['Sharpe'] = df_display['Sharpe'].apply(lambda x: f"{x:.3f}")
    df_display['Beta'] = df_display['Beta'].apply(lambda x: f"{x:.2f}")
    df_display['Score'] = df_display['Score'].apply(lambda x: f"{x:.4f}")
    
    display(df_display)
    
    # Estadísticas agregadas
    print(f"\n{'─'*70}")
    print("📊 ESTADÍSTICAS AGREGADAS DEL PORTAFOLIO:")
    print(f"{'─'*70}")
    print(f"   • Retorno Anual Promedio: {df_portafolio['return_annualized'].mean()*100:.2f}%")
    print(f"   • Momentum 6m Promedio: {df_portafolio['momentum_6m'].mean()*100:.2f}%")
    print(f"   • Sharpe Ratio Promedio: {df_portafolio['sharpe_ratio'].mean():.3f}")
    print(f"   • Beta Promedio: {df_portafolio['beta'].mean():.2f} {'🚀 (>1 amplifica ganancias)' if df_portafolio['beta'].mean() > 1 else '🛡️ (<1 reduce volatilidad)'}")
    print(f"   • Volatilidad Promedio: {df_portafolio['volatility_annual'].mean()*100:.2f}%")
    print(f"   • Score Momentum Promedio: {df_portafolio['score_compuesto'].mean():.4f}")
    
    # Distribución por segmento
    print(f"\n{'─'*70}")
    print("📈 DISTRIBUCIÓN POR SEGMENTO:")
    dist = df_portafolio.groupby('segmento_nombre').size()
    for segmento, count in dist.items():
        print(f"   • {segmento}: {count} activos ({count*10}%)")

print("✅ Funciones de visualización definidas")

✅ Funciones de visualización definidas


## 6. 🎯 Selección Interactiva del Perfil de Inversión

In [6]:
# Variables globales para almacenar la selección
portafolio_actual = None

# Crear widget de selección
perfil_dropdown = widgets.Dropdown(
    options=list(PERFILES_INVERSION.keys()),
    value='Moderado',
    description='Perfil:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='250px')
)

# Botón para ejecutar
boton_seleccionar = widgets.Button(
    description='🎯 Seleccionar Portafolio',
    button_style='success',
    layout=widgets.Layout(width='200px', height='35px')
)

# Output para mostrar resultados
output = widgets.Output()

def on_button_click(b):
    global portafolio_actual
    
    with output:
        clear_output(wait=True)
        
        perfil = perfil_dropdown.value
        print(f"\n🔄 Seleccionando portafolio para perfil: {perfil}...\n")
        
        # Seleccionar portafolio
        portafolio_actual = seleccionar_portafolio(perfil, df_segmentacion)
        
        # Mostrar resumen
        mostrar_resumen_portafolio(portafolio_actual, perfil)
        
        # Obtener tickers
        tickers = portafolio_actual['ticker'].tolist()
        
        # Graficar precios normalizados
        print(f"\n{'='*70}")
        print("📈 EVOLUCIÓN DE PRECIOS NORMALIZADOS")
        print(f"{'='*70}")
        
        fig = graficar_precios_normalizados(
            tickers, 
            df_precios, 
            f"Portafolio {perfil} - Precios Normalizados (Base 100)"
        )
        
        if fig:
            fig.show()

boton_seleccionar.on_click(on_button_click)

# Mostrar interfaz
print("🎛️ SELECTOR DE PERFIL DE INVERSIÓN")
print("="*50)
print("Seleccione su perfil de inversión y haga clic en el botón para")
print("generar el portafolio de 10 activos optimizado.\n")

display(widgets.HBox([perfil_dropdown, boton_seleccionar]))
display(output)

🎛️ SELECTOR DE PERFIL DE INVERSIÓN
Seleccione su perfil de inversión y haga clic en el botón para
generar el portafolio de 10 activos optimizado.



Output()

## 6.1 Generar Portafolio para Análisis

Esta celda genera el portafolio basado en el perfil seleccionado en el widget anterior. Este portafolio se utilizará en las secciones de backtesting y análisis posteriores.

In [40]:
# Tomar el perfil seleccionado del dropdown
perfil = perfil_dropdown.value

print(f"🎯 Perfil seleccionado: {perfil}")
print(f"📝 {PERFILES_INVERSION[perfil]['descripcion']}")
print(f"\n📊 FÓRMULA DE SELECCIÓN: Score Momentum (CORREGIDA)")
print(f"   0.35×Return + 0.30×Momentum_6m + 0.15×Sharpe + 0.20×Beta")
print(f"   ⚠️ Beta alto ahora es POSITIVO (amplifica ganancias en mercados alcistas)\n")

# Seleccionar portafolio
portafolio_actual = seleccionar_portafolio(perfil, df_segmentacion)

# Mostrar resumen
mostrar_resumen_portafolio(portafolio_actual, perfil)

# Obtener tickers
tickers = portafolio_actual['ticker'].tolist()

# Graficar precios normalizados
print(f"\n{'='*70}")
print("📈 EVOLUCIÓN DE PRECIOS NORMALIZADOS")
print(f"{'='*70}")

fig = graficar_precios_normalizados(
    tickers, 
    df_precios, 
    f"Portafolio {perfil} - Precios Normalizados (Base 100)"
)

if fig:
    fig.show()

🎯 Perfil seleccionado: Moderado
📝 Balance entre crecimiento y estabilidad

📊 FÓRMULA DE SELECCIÓN: Score Momentum (CORREGIDA)
   0.35×Return + 0.30×Momentum_6m + 0.15×Sharpe + 0.20×Beta
   ⚠️ Beta alto ahora es POSITIVO (amplifica ganancias en mercados alcistas)


📊 PORTAFOLIO SELECCIONADO - PERFIL: MODERADO
📝 Descripción: Balance entre crecimiento y estabilidad
📈 Clusters: Alto Rendimiento (C1), Moderado (C2), Estable (C3)

──────────────────────────────────────────────────────────────────────


,Ticker,Segmento,Retorno Anual,Momentum 6m,Sharpe,Beta,Score,Peso
0,NUE,Alto_Rendimiento,49.47%,5.09%,1.066,1.19,0.6607,10.0%
1,PWR,Alto_Rendimiento,47.12%,10.27%,1.178,1.08,0.6562,10.0%
2,IT,Alto_Rendimiento,42.15%,29.97%,1.119,1.09,0.6495,10.0%
3,OXY,Alto_Rendimiento,51.07%,0.84%,0.984,0.89,0.6488,10.0%
4,AMAT,Moderado,23.26%,11.41%,0.583,1.77,0.6020,10.0%
5,RCL,Moderado,22.49%,25.47%,0.553,1.63,0.5985,10.0%
6,AMD,Moderado,17.00%,27.28%,0.464,1.96,0.5954,10.0%
7,GS,Estable,13.42%,18.11%,0.419,0.98,0.5417,10.0%
8,BRO,Estable,15.44%,5.47%,0.501,0.90,0.5414,10.0%
9,NRG,Estable,12.59%,37.43%,0.374,0.84,0.5400,10.0%



──────────────────────────────────────────────────────────────────────
📊 ESTADÍSTICAS AGREGADAS DEL PORTAFOLIO:
──────────────────────────────────────────────────────────────────────
   • Retorno Anual Promedio: 29.40%
   • Momentum 6m Promedio: 17.13%
   • Sharpe Ratio Promedio: 0.724
   • Beta Promedio: 1.23 🚀 (>1 amplifica ganancias)
   • Volatilidad Promedio: 38.61%
   • Score Momentum Promedio: 0.6034

──────────────────────────────────────────────────────────────────────
📈 DISTRIBUCIÓN POR SEGMENTO:
   • Alto_Rendimiento: 4 activos (40%)
   • Estable: 3 activos (30%)
   • Moderado: 3 activos (30%)

📈 EVOLUCIÓN DE PRECIOS NORMALIZADOS


## 7. Exportar Portafolio Seleccionado (Opcional)

In [41]:
# Ejecutar esta celda después de seleccionar un portafolio para exportarlo
if portafolio_actual is not None:
    # Guardar el portafolio seleccionado
    perfil = perfil_dropdown.value
    output_path = f'../reports/portafolio_{perfil.lower()}.csv'
    portafolio_actual.to_csv(output_path, index=False)
    print(f"✅ Portafolio exportado a: {output_path}")
    print(f"\n📋 Tickers del portafolio {perfil}:")
    print(portafolio_actual['ticker'].tolist())
else:
    print("⚠️ Primero seleccione un portafolio usando el widget de arriba")

✅ Portafolio exportado a: ../reports/portafolio_moderado.csv

📋 Tickers del portafolio Moderado:
['NUE', 'PWR', 'IT', 'OXY', 'AMAT', 'RCL', 'AMD', 'GS', 'BRO', 'NRG']


## 8. Backtesting (Benchmark)

### 8.1 Configuración de Parámetros del Backtesting

**Costos de transacción realistas (basados en brokers retail de EE.UU.):**
- **Comisión por operación**: $0 (brokers como Fidelity, Schwab, Robinhood)
- **SEC Fee**: ~$0.0000229 por dólar vendido (~2.29 bps)
- **FINRA TAF**: $0.000145 por acción (máx $7.27)
- **Spread bid-ask estimado**: ~0.05% - 0.10%

**Costo total estimado**: ~0.10% (10 basis points) por transacción de compra + venta

In [42]:
# ============================================================================
# CONFIGURACIÓN DE PARÁMETROS DEL BACKTESTING
# ============================================================================

# Cargar datos de precios de test
PATH_PRECIOS_TEST = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\data\prices_test.csv'
df_precios_test = pd.read_csv(PATH_PRECIOS_TEST, parse_dates=['date'], index_col='date')

# Parámetros de la simulación
CAPITAL_INICIAL = 10_000  # USD $10,000
BENCHMARK_TICKER = 'SPY'

# Costos de transacción realistas (investigación de mercado)
# Basado en brokers retail de EE.UU. (Fidelity, Schwab, TD Ameritrade)
COSTOS_TRANSACCION = {
    'comision_por_orden': 0.00,          # $0 en la mayoría de brokers modernos
    'sec_fee_rate': 0.0000229,            # SEC Fee: $22.90 por $1M vendidos
    'finra_taf_rate': 0.000145,           # FINRA TAF: $0.000145 por acción
    'spread_estimado': 0.0005,            # Spread bid-ask: ~0.05% (5 bps)
    'slippage_estimado': 0.0005,          # Slippage: ~0.05% (5 bps)
}

# Costo total por transacción (entrada + salida)
# ~10 basis points (0.10%) es realista para inversiones retail
COSTO_TOTAL_ROUNDTRIP = 0.0010  # 0.10% = 10 bps (compra + venta)

# Fechas del backtest
FECHA_INICIO = df_precios_test.index.min()
FECHA_FIN = df_precios_test.index.max()

print("=" * 70)
print("📊 CONFIGURACIÓN DEL BACKTESTING")
print("=" * 70)

# Mostrar el perfil de inversión seleccionado
if 'perfil' in dir() and perfil is not None:
    print(f"\n🎯 PERFIL DE INVERSIÓN: {perfil.upper()}")
    print(f"   📝 {PERFILES_INVERSION[perfil]['descripcion']}")

print(f"\n💰 Capital Inicial: ${CAPITAL_INICIAL:,.2f}")
print(f"📈 Benchmark: {BENCHMARK_TICKER}")
print(f"📅 Período: {FECHA_INICIO.strftime('%Y-%m-%d')} a {FECHA_FIN.strftime('%Y-%m-%d')}")
print(f"📆 Días de trading: {len(df_precios_test)} días")
print(f"\n💸 Costos de Transacción:")
print(f"   • Comisión por orden: ${COSTOS_TRANSACCION['comision_por_orden']:.2f}")
print(f"   • SEC Fee: {COSTOS_TRANSACCION['sec_fee_rate']*100:.4f}%")
print(f"   • FINRA TAF: ${COSTOS_TRANSACCION['finra_taf_rate']:.6f}/acción")
print(f"   • Spread estimado: {COSTOS_TRANSACCION['spread_estimado']*100:.2f}%")
print(f"   • Slippage estimado: {COSTOS_TRANSACCION['slippage_estimado']*100:.2f}%")
print(f"   • 🔹 Costo total round-trip: {COSTO_TOTAL_ROUNDTRIP*100:.2f}% ({COSTO_TOTAL_ROUNDTRIP*10000:.0f} bps)")

# Verificar que los tickers del portafolio están disponibles
if portafolio_actual is not None:
    tickers_portafolio = portafolio_actual['ticker'].tolist()
    tickers_disponibles = [t for t in tickers_portafolio if t in df_precios_test.columns]
    tickers_faltantes = [t for t in tickers_portafolio if t not in df_precios_test.columns]
    
    print(f"\n📋 Portafolio seleccionado: {len(tickers_portafolio)} activos")
    print(f"   🏷️ Tickers: {', '.join(tickers_portafolio)}")
    print(f"   ✅ Disponibles en test: {len(tickers_disponibles)}")
    if tickers_faltantes:
        print(f"   ⚠️ No disponibles: {tickers_faltantes}")
else:
    print("\n⚠️ No hay portafolio seleccionado. Ejecuta las celdas anteriores primero.")

📊 CONFIGURACIÓN DEL BACKTESTING

🎯 PERFIL DE INVERSIÓN: MODERADO
   📝 Balance entre crecimiento y estabilidad

💰 Capital Inicial: $10,000.00
📈 Benchmark: SPY
📅 Período: 2024-01-02 a 2025-12-19
📆 Días de trading: 495 días

💸 Costos de Transacción:
   • Comisión por orden: $0.00
   • SEC Fee: 0.0023%
   • FINRA TAF: $0.000145/acción
   • Spread estimado: 0.05%
   • Slippage estimado: 0.05%
   • 🔹 Costo total round-trip: 0.10% (10 bps)

📋 Portafolio seleccionado: 10 activos
   🏷️ Tickers: NUE, PWR, IT, OXY, AMAT, RCL, AMD, GS, BRO, NRG
   ✅ Disponibles en test: 10


### 8.2 Simulación de Estrategia Buy & Hold

**Estrategia Simple:**
- Comprar todos los activos del portafolio al inicio del período (equi-ponderado)
- Mantener hasta el final del período (sin rebalanceo)
- Vender todos los activos al final

**Comparación con Benchmark (SPY):**
- Misma estrategia Buy & Hold con el capital completo en SPY

In [43]:
# ============================================================================
# SIMULACIÓN DE ESTRATEGIA BUY & HOLD
# ============================================================================

def simular_buy_and_hold(df_precios, tickers, capital_inicial, costo_roundtrip, pesos=None):
    """
    Simula una estrategia Buy & Hold con costos de transacción.
    
    Args:
        df_precios: DataFrame con precios históricos
        tickers: Lista de tickers a comprar
        capital_inicial: Capital inicial en USD
        costo_roundtrip: Costo total de transacción (compra + venta)
        pesos: Diccionario con pesos por ticker (opcional, default equi-ponderado)
    
    Returns:
        dict con resultados del backtest
    """
    # Filtrar tickers disponibles
    tickers_disponibles = [t for t in tickers if t in df_precios.columns]
    
    if len(tickers_disponibles) == 0:
        raise ValueError("No hay tickers disponibles en los datos de precios")
    
    # Definir pesos (equi-ponderado si no se especifica)
    if pesos is None:
        peso_por_activo = 1 / len(tickers_disponibles)
        pesos = {t: peso_por_activo for t in tickers_disponibles}
    
    # ==========================================
    # FASE 1: COMPRA INICIAL
    # ==========================================
    
    precios_iniciales = df_precios[tickers_disponibles].iloc[0]
    
    # Capital disponible después de costos de entrada (mitad del roundtrip)
    costo_entrada = costo_roundtrip / 2
    capital_despues_costos = capital_inicial * (1 - costo_entrada)
    
    # Calcular posiciones (número de acciones por activo)
    posiciones = {}
    valor_invertido = {}
    
    for ticker in tickers_disponibles:
        capital_asignado = capital_despues_costos * pesos[ticker]
        precio_compra = precios_iniciales[ticker]
        
        # Número de acciones (permitimos fracciones para simplicidad)
        n_acciones = capital_asignado / precio_compra
        posiciones[ticker] = n_acciones
        valor_invertido[ticker] = capital_asignado
    
    # ==========================================
    # FASE 2: CÁLCULO DEL EQUITY CURVE DIARIO
    # ==========================================
    
    # Calcular valor del portafolio cada día
    equity_curve = pd.Series(index=df_precios.index, dtype=float)
    
    for fecha in df_precios.index:
        valor_dia = 0
        for ticker in tickers_disponibles:
            precio_dia = df_precios.loc[fecha, ticker]
            valor_dia += posiciones[ticker] * precio_dia
        equity_curve[fecha] = valor_dia
    
    # ==========================================
    # FASE 3: VENTA FINAL (aplicar costos de salida)
    # ==========================================
    
    costo_salida = costo_roundtrip / 2
    valor_final_bruto = equity_curve.iloc[-1]
    valor_final_neto = valor_final_bruto * (1 - costo_salida)
    
    # Ajustar el último valor del equity curve
    equity_curve.iloc[-1] = valor_final_neto
    
    # ==========================================
    # FASE 4: CALCULAR MÉTRICAS
    # ==========================================
    
    # Retornos diarios
    retornos_diarios = equity_curve.pct_change().dropna()
    
    # Métricas de rendimiento
    retorno_total = (valor_final_neto - capital_inicial) / capital_inicial
    dias_trading = len(df_precios)
    retorno_anualizado = (1 + retorno_total) ** (252 / dias_trading) - 1
    
    # Volatilidad
    volatilidad_diaria = retornos_diarios.std()
    volatilidad_anualizada = volatilidad_diaria * np.sqrt(252)
    
    # Sharpe Ratio (asumiendo risk-free rate = 4.5% anual para 2024)
    risk_free_rate = 0.045
    sharpe_ratio = (retorno_anualizado - risk_free_rate) / volatilidad_anualizada if volatilidad_anualizada > 0 else 0
    
    # Sortino Ratio
    retornos_negativos = retornos_diarios[retornos_diarios < 0]
    downside_std = retornos_negativos.std() * np.sqrt(252) if len(retornos_negativos) > 0 else 0
    sortino_ratio = (retorno_anualizado - risk_free_rate) / downside_std if downside_std > 0 else 0
    
    # Maximum Drawdown
    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    # Calmar Ratio
    calmar_ratio = retorno_anualizado / abs(max_drawdown) if max_drawdown != 0 else 0
    
    return {
        'equity_curve': equity_curve,
        'drawdown': drawdown,
        'retornos_diarios': retornos_diarios,
        'capital_inicial': capital_inicial,
        'valor_final': valor_final_neto,
        'retorno_total': retorno_total,
        'retorno_anualizado': retorno_anualizado,
        'volatilidad_anualizada': volatilidad_anualizada,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'max_drawdown': max_drawdown,
        'calmar_ratio': calmar_ratio,
        'posiciones': posiciones,
        'tickers': tickers_disponibles,
        'dias_trading': dias_trading,
        'costo_total_aplicado': capital_inicial * costo_roundtrip
    }

# ==========================================
# EJECUTAR SIMULACIÓN
# ==========================================

if portafolio_actual is not None:
    # Simular portafolio
    print("🔄 Simulando estrategia Buy & Hold para el portafolio...")
    resultados_portafolio = simular_buy_and_hold(
        df_precios=df_precios_test,
        tickers=portafolio_actual['ticker'].tolist(),
        capital_inicial=CAPITAL_INICIAL,
        costo_roundtrip=COSTO_TOTAL_ROUNDTRIP
    )
    
    # Simular benchmark (SPY)
    print("🔄 Simulando estrategia Buy & Hold para el benchmark (SPY)...")
    resultados_benchmark = simular_buy_and_hold(
        df_precios=df_precios_test,
        tickers=[BENCHMARK_TICKER],
        capital_inicial=CAPITAL_INICIAL,
        costo_roundtrip=COSTO_TOTAL_ROUNDTRIP
    )
    
    print("✅ Simulaciones completadas")
else:
    print("⚠️ No hay portafolio seleccionado. Ejecuta las celdas anteriores primero.")

🔄 Simulando estrategia Buy & Hold para el portafolio...
🔄 Simulando estrategia Buy & Hold para el benchmark (SPY)...
✅ Simulaciones completadas


### 8.3 Equity Curves y Drawdown

In [44]:
# ============================================================================
# VISUALIZACIÓN DE EQUITY CURVES Y DRAWDOWN
# ============================================================================

def graficar_backtest_resultados(resultados_portafolio, resultados_benchmark, perfil_nombre):
    """
    Genera gráficos de equity curve y drawdown comparativos.
    """
    from plotly.subplots import make_subplots
    
    # Crear figura con 2 subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=(
            f'💰 Equity Curve: Portafolio {perfil_nombre} vs SPY (Capital Inicial: $10,000)',
            '📉 Drawdown Comparativo'
        ),
        row_heights=[0.6, 0.4]
    )
    
    # ==========================================
    # GRÁFICO 1: EQUITY CURVES
    # ==========================================
    
    # Equity del portafolio
    fig.add_trace(
        go.Scatter(
            x=resultados_portafolio['equity_curve'].index,
            y=resultados_portafolio['equity_curve'].values,
            mode='lines',
            name=f'Portafolio {perfil_nombre}',
            line=dict(color='#2E86AB', width=2),
            hovertemplate='Portafolio<br>Fecha: %{x}<br>Valor: $%{y:,.2f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Equity del benchmark
    fig.add_trace(
        go.Scatter(
            x=resultados_benchmark['equity_curve'].index,
            y=resultados_benchmark['equity_curve'].values,
            mode='lines',
            name='SPY (Benchmark)',
            line=dict(color='#E94F37', width=2, dash='dot'),
            hovertemplate='SPY<br>Fecha: %{x}<br>Valor: $%{y:,.2f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Línea de capital inicial
    fig.add_hline(
        y=CAPITAL_INICIAL, 
        line_dash="dash", 
        line_color="gray", 
        opacity=0.5,
        annotation_text=f"Capital Inicial: ${CAPITAL_INICIAL:,}",
        annotation_position="right",
        row=1, col=1
    )
    
    # ==========================================
    # GRÁFICO 2: DRAWDOWN
    # ==========================================
    
    # Drawdown del portafolio
    fig.add_trace(
        go.Scatter(
            x=resultados_portafolio['drawdown'].index,
            y=resultados_portafolio['drawdown'].values * 100,
            mode='lines',
            name=f'DD Portafolio',
            fill='tozeroy',
            line=dict(color='#2E86AB', width=1),
            fillcolor='rgba(46, 134, 171, 0.3)',
            hovertemplate='Portafolio DD<br>Fecha: %{x}<br>Drawdown: %{y:.2f}%<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Drawdown del benchmark
    fig.add_trace(
        go.Scatter(
            x=resultados_benchmark['drawdown'].index,
            y=resultados_benchmark['drawdown'].values * 100,
            mode='lines',
            name='DD SPY',
            line=dict(color='#E94F37', width=1, dash='dot'),
            hovertemplate='SPY DD<br>Fecha: %{x}<br>Drawdown: %{y:.2f}%<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Layout
    fig.update_layout(
        height=700,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=100)
    )
    
    fig.update_yaxes(title_text="Valor ($)", row=1, col=1, tickprefix="$", tickformat=",")
    fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1, ticksuffix="%")
    fig.update_xaxes(title_text="Fecha", row=2, col=1)
    
    return fig

# Ejecutar visualización
if 'resultados_portafolio' in dir() and resultados_portafolio is not None:
    fig_backtest = graficar_backtest_resultados(
        resultados_portafolio, 
        resultados_benchmark,
        perfil
    )
    fig_backtest.show()
else:
    print("⚠️ Ejecuta la celda de simulación primero.")

### 8.4 Métricas Comparativas del Backtesting

In [45]:
# ============================================================================
# MÉTRICAS COMPARATIVAS DEL BACKTESTING
# ============================================================================

def calcular_metricas_mensuales(equity_curve):
    """Calcula retornos mensuales a partir del equity curve."""
    # Resample a fin de mes
    equity_mensual = equity_curve.resample('ME').last()
    retornos_mensuales = equity_mensual.pct_change().dropna()
    return retornos_mensuales

def generar_tabla_metricas(resultados_portafolio, resultados_benchmark, perfil_nombre):
    """Genera una tabla comparativa de métricas."""
    
    metricas = {
        'Métrica': [
            '💰 Capital Inicial',
            '💵 Valor Final',
            '📈 Retorno Total',
            '📊 Retorno Anualizado',
            '📉 Volatilidad Anualizada',
            '⚖️ Sharpe Ratio',
            '🎯 Sortino Ratio',
            '🔻 Maximum Drawdown',
            '📐 Calmar Ratio',
            '📆 Días de Trading',
            '💸 Costos Aplicados'
        ],
        f'Portafolio {perfil_nombre}': [
            f"${resultados_portafolio['capital_inicial']:,.2f}",
            f"${resultados_portafolio['valor_final']:,.2f}",
            f"{resultados_portafolio['retorno_total']*100:.2f}%",
            f"{resultados_portafolio['retorno_anualizado']*100:.2f}%",
            f"{resultados_portafolio['volatilidad_anualizada']*100:.2f}%",
            f"{resultados_portafolio['sharpe_ratio']:.3f}",
            f"{resultados_portafolio['sortino_ratio']:.3f}",
            f"{resultados_portafolio['max_drawdown']*100:.2f}%",
            f"{resultados_portafolio['calmar_ratio']:.3f}",
            f"{resultados_portafolio['dias_trading']}",
            f"${resultados_portafolio['costo_total_aplicado']:.2f}"
        ],
        'SPY (Benchmark)': [
            f"${resultados_benchmark['capital_inicial']:,.2f}",
            f"${resultados_benchmark['valor_final']:,.2f}",
            f"{resultados_benchmark['retorno_total']*100:.2f}%",
            f"{resultados_benchmark['retorno_anualizado']*100:.2f}%",
            f"{resultados_benchmark['volatilidad_anualizada']*100:.2f}%",
            f"{resultados_benchmark['sharpe_ratio']:.3f}",
            f"{resultados_benchmark['sortino_ratio']:.3f}",
            f"{resultados_benchmark['max_drawdown']*100:.2f}%",
            f"{resultados_benchmark['calmar_ratio']:.3f}",
            f"{resultados_benchmark['dias_trading']}",
            f"${resultados_benchmark['costo_total_aplicado']:.2f}"
        ]
    }
    
    # Calcular diferencias (Alpha)
    metricas['Diferencia (Alpha)'] = [
        '-',
        f"${resultados_portafolio['valor_final'] - resultados_benchmark['valor_final']:+,.2f}",
        f"{(resultados_portafolio['retorno_total'] - resultados_benchmark['retorno_total'])*100:+.2f}%",
        f"{(resultados_portafolio['retorno_anualizado'] - resultados_benchmark['retorno_anualizado'])*100:+.2f}%",
        f"{(resultados_portafolio['volatilidad_anualizada'] - resultados_benchmark['volatilidad_anualizada'])*100:+.2f}%",
        f"{resultados_portafolio['sharpe_ratio'] - resultados_benchmark['sharpe_ratio']:+.3f}",
        f"{resultados_portafolio['sortino_ratio'] - resultados_benchmark['sortino_ratio']:+.3f}",
        f"{(resultados_portafolio['max_drawdown'] - resultados_benchmark['max_drawdown'])*100:+.2f}%",
        f"{resultados_portafolio['calmar_ratio'] - resultados_benchmark['calmar_ratio']:+.3f}",
        '-',
        '-'
    ]
    
    return pd.DataFrame(metricas)

def generar_metricas_mensuales_comparativas(resultados_portafolio, resultados_benchmark):
    """Genera tabla de retornos mensuales comparativos."""
    
    ret_mensual_port = calcular_metricas_mensuales(resultados_portafolio['equity_curve'])
    ret_mensual_bench = calcular_metricas_mensuales(resultados_benchmark['equity_curve'])
    
    # Crear DataFrame
    df_mensual = pd.DataFrame({
        'Mes': ret_mensual_port.index.strftime('%Y-%m'),
        'Portafolio': ret_mensual_port.values * 100,
        'SPY': ret_mensual_bench.values * 100,
    })
    df_mensual['Diferencia'] = df_mensual['Portafolio'] - df_mensual['SPY']
    
    # Formatear
    df_mensual['Portafolio'] = df_mensual['Portafolio'].apply(lambda x: f"{x:.2f}%")
    df_mensual['SPY'] = df_mensual['SPY'].apply(lambda x: f"{x:.2f}%")
    df_mensual['Diferencia'] = df_mensual['Diferencia'].apply(lambda x: f"{x:+.2f}%")
    
    return df_mensual

# ==========================================
# MOSTRAR MÉTRICAS
# ==========================================

if 'resultados_portafolio' in dir() and resultados_portafolio is not None:
    print("=" * 80)
    print("📊 MÉTRICAS COMPARATIVAS: PORTAFOLIO vs BENCHMARK (SPY)")
    print("=" * 80)
    
    # Tabla principal de métricas
    df_metricas = generar_tabla_metricas(resultados_portafolio, resultados_benchmark, perfil)
    display(df_metricas.style.set_properties(**{'text-align': 'left'}).hide(axis='index'))
    
    # Evaluación del desempeño
    print("\n" + "=" * 80)
    print("📝 EVALUACIÓN DEL DESEMPEÑO")
    print("=" * 80)
    
    alpha = resultados_portafolio['retorno_total'] - resultados_benchmark['retorno_total']
    if alpha > 0:
        print(f"✅ El portafolio SUPERÓ al benchmark por {alpha*100:.2f} puntos porcentuales")
    else:
        print(f"❌ El portafolio TUVO UN DESEMPEÑO INFERIOR al benchmark por {abs(alpha)*100:.2f} puntos porcentuales")
    
    # Comparación de Sharpe
    sharpe_diff = resultados_portafolio['sharpe_ratio'] - resultados_benchmark['sharpe_ratio']
    if sharpe_diff > 0:
        print(f"✅ Mejor relación riesgo-retorno (Sharpe +{sharpe_diff:.3f})")
    else:
        print(f"⚠️ Menor relación riesgo-retorno (Sharpe {sharpe_diff:.3f})")
    
    # Comparación de Drawdown
    dd_diff = resultados_portafolio['max_drawdown'] - resultados_benchmark['max_drawdown']
    if dd_diff > 0:  # Menos negativo es mejor
        print(f"✅ Mejor protección a la baja (Max DD +{dd_diff*100:.2f}%)")
    else:
        print(f"⚠️ Mayor exposición a caídas (Max DD {dd_diff*100:.2f}%)")
    
    # Tabla de retornos mensuales
    print("\n" + "=" * 80)
    print("📅 RETORNOS MENSUALES COMPARATIVOS")
    print("=" * 80)
    df_mensual = generar_metricas_mensuales_comparativas(resultados_portafolio, resultados_benchmark)
    display(df_mensual)
    
    # ==========================================
    # RENDIMIENTO INDIVIDUAL DE CADA ACTIVO
    # ==========================================
    print("\n" + "=" * 80)
    print("📈 RENDIMIENTO INDIVIDUAL DE CADA ACTIVO")
    print("=" * 80)
    
    # Calcular rendimiento de cada activo
    tickers_port = resultados_portafolio['tickers']
    rendimientos_individuales = []
    
    for ticker in tickers_port:
        if ticker in df_precios_test.columns:
            precio_inicio = df_precios_test[ticker].iloc[0]
            precio_fin = df_precios_test[ticker].iloc[-1]
            retorno = (precio_fin - precio_inicio) / precio_inicio
            
            # Calcular volatilidad y max drawdown individual
            precios_ticker = df_precios_test[ticker]
            retornos_diarios = precios_ticker.pct_change().dropna()
            volatilidad = retornos_diarios.std() * np.sqrt(252)
            
            rolling_max = precios_ticker.cummax()
            drawdown = (precios_ticker - rolling_max) / rolling_max
            max_dd = drawdown.min()
            
            # Obtener segmento del activo
            segmento = portafolio_actual[portafolio_actual['ticker'] == ticker]['segmento_nombre'].values
            segmento = segmento[0] if len(segmento) > 0 else 'N/A'
            
            rendimientos_individuales.append({
                'Ticker': ticker,
                'Segmento': segmento,
                'Retorno Total': retorno,
                'Volatilidad': volatilidad,
                'Max Drawdown': max_dd,
                'vs SPY': retorno - resultados_benchmark['retorno_total']
            })
    
    # Crear DataFrame y ordenar por retorno
    df_rendimientos = pd.DataFrame(rendimientos_individuales)
    df_rendimientos = df_rendimientos.sort_values('Retorno Total', ascending=False)
    
    # Formatear para display
    df_rend_display = df_rendimientos.copy()
    df_rend_display['Retorno Total'] = df_rend_display['Retorno Total'].apply(lambda x: f"{x*100:.2f}%")
    df_rend_display['Volatilidad'] = df_rend_display['Volatilidad'].apply(lambda x: f"{x*100:.2f}%")
    df_rend_display['Max Drawdown'] = df_rend_display['Max Drawdown'].apply(lambda x: f"{x*100:.2f}%")
    df_rend_display['vs SPY'] = df_rend_display['vs SPY'].apply(lambda x: f"{x*100:+.2f}%")
    
    display(df_rend_display)
    
    # Estadísticas resumidas
    print(f"\n📊 ESTADÍSTICAS DE LOS ACTIVOS:")
    print(f"   • Mejor activo: {df_rendimientos.iloc[0]['Ticker']} ({df_rendimientos.iloc[0]['Retorno Total']*100:.2f}%)")
    print(f"   • Peor activo: {df_rendimientos.iloc[-1]['Ticker']} ({df_rendimientos.iloc[-1]['Retorno Total']*100:.2f}%)")
    
    activos_superan_spy = df_rendimientos[df_rendimientos['vs SPY'] > 0]
    print(f"   • Activos que superan SPY: {len(activos_superan_spy)}/{len(df_rendimientos)}")
    print(f"   • Retorno promedio activos: {df_rendimientos['Retorno Total'].mean()*100:.2f}%")
    print(f"   • Retorno SPY: {resultados_benchmark['retorno_total']*100:.2f}%")
    
else:
    print("⚠️ Ejecuta las celdas anteriores primero.")

📊 MÉTRICAS COMPARATIVAS: PORTAFOLIO vs BENCHMARK (SPY)


Métrica,Portafolio Moderado,SPY (Benchmark),Diferencia (Alpha)
💰 Capital Inicial,"$10,000.00","$10,000.00",-
💵 Valor Final,"$16,322.56","$14,383.79","$+1,938.78"
📈 Retorno Total,63.23%,43.84%,+19.39%
📊 Retorno Anualizado,28.33%,20.33%,+8.00%
📉 Volatilidad Anualizada,24.74%,16.47%,+8.26%
⚖️ Sharpe Ratio,0.963,0.961,+0.002
🎯 Sortino Ratio,1.319,1.227,+0.092
🔻 Maximum Drawdown,-24.02%,-19.00%,-5.03%
📐 Calmar Ratio,1.179,1.070,+0.109
📆 Días de Trading,495,495,-



📝 EVALUACIÓN DEL DESEMPEÑO
✅ El portafolio SUPERÓ al benchmark por 19.39 puntos porcentuales
✅ Mejor relación riesgo-retorno (Sharpe +0.002)
⚠️ Mayor exposición a caídas (Max DD -5.03%)

📅 RETORNOS MENSUALES COMPARATIVOS


,Mes,Portafolio,SPY,Diferencia
0,2024-02,8.27%,5.22%,+3.05%
1,2024-03,5.77%,2.95%,+2.82%
2,2024-04,-3.91%,-4.03%,+0.12%
3,2024-05,5.45%,5.06%,+0.40%
4,2024-06,0.30%,3.20%,-2.90%
5,2024-07,0.78%,1.21%,-0.43%
6,2024-08,1.41%,2.34%,-0.93%
7,2024-09,2.83%,1.79%,+1.04%
8,2024-10,-0.33%,-0.89%,+0.57%
9,2024-11,8.73%,5.96%,+2.77%



📈 RENDIMIENTO INDIVIDUAL DE CADA ACTIVO


,Ticker,Segmento,Retorno Total,Volatilidad,Max Drawdown,vs SPY
9,NRG,Estable,205.42%,46.88%,-26.22%,+161.59%
5,RCL,Moderado,145.86%,39.80%,-35.25%,+102.03%
7,GS,Estable,130.35%,28.74%,-31.24%,+86.51%
1,PWR,Alto_Rendimiento,103.97%,36.61%,-33.89%,+60.13%
4,AMAT,Moderado,66.23%,44.21%,-50.21%,+22.39%
6,AMD,Moderado,54.37%,54.88%,-63.00%,+10.53%
8,BRO,Estable,13.78%,22.01%,-38.58%,-30.06%
0,NUE,Alto_Rendimiento,-9.92%,35.30%,-48.64%,-53.76%
3,OXY,Alto_Rendimiento,-33.86%,30.07%,-47.86%,-77.69%
2,IT,Alto_Rendimiento,-42.32%,33.64%,-59.38%,-86.15%



📊 ESTADÍSTICAS DE LOS ACTIVOS:
   • Mejor activo: NRG (205.42%)
   • Peor activo: IT (-42.32%)
   • Activos que superan SPY: 6/10
   • Retorno promedio activos: 63.39%
   • Retorno SPY: 43.84%


### 8.5 Gráfico de Retornos Mensuales Comparativos

In [31]:
# ============================================================================
# GRÁFICO DE RETORNOS MENSUALES COMPARATIVOS
# ============================================================================

def graficar_retornos_mensuales(resultados_portafolio, resultados_benchmark, perfil_nombre):
    """Genera un gráfico de barras comparativo de retornos mensuales."""
    
    ret_mensual_port = calcular_metricas_mensuales(resultados_portafolio['equity_curve'])
    ret_mensual_bench = calcular_metricas_mensuales(resultados_benchmark['equity_curve'])
    
    meses = ret_mensual_port.index.strftime('%b %Y')
    
    fig = go.Figure()
    
    # Barras del portafolio
    fig.add_trace(go.Bar(
        name=f'Portafolio {perfil_nombre}',
        x=meses,
        y=ret_mensual_port.values * 100,
        marker_color='#2E86AB',
        hovertemplate='Portafolio<br>%{x}<br>Retorno: %{y:.2f}%<extra></extra>'
    ))
    
    # Barras del benchmark
    fig.add_trace(go.Bar(
        name='SPY (Benchmark)',
        x=meses,
        y=ret_mensual_bench.values * 100,
        marker_color='#E94F37',
        hovertemplate='SPY<br>%{x}<br>Retorno: %{y:.2f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'📊 Retornos Mensuales: Portafolio {perfil_nombre} vs SPY',
        xaxis_title='Mes',
        yaxis_title='Retorno (%)',
        barmode='group',
        template='plotly_white',
        height=450,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )
    
    fig.update_yaxes(ticksuffix='%')
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    return fig

# Ejecutar
if 'resultados_portafolio' in dir() and resultados_portafolio is not None:
    fig_mensual = graficar_retornos_mensuales(resultados_portafolio, resultados_benchmark, perfil)
    fig_mensual.show()
else:
    print("⚠️ Ejecuta las celdas anteriores primero.")

### 8.6 Comparación Anual (2024 vs 2025)

Análisis del rendimiento del portafolio y benchmark desglosado por año calendario.

In [32]:
# ============================================================================
# COMPARACIÓN ANUAL (2024 vs 2025)
# ============================================================================

def calcular_metricas_anuales(equity_curve):
    """Calcula retornos anuales a partir del equity curve."""
    # Obtener el primer y último valor de cada año
    metricas_por_anio = {}
    
    for year in equity_curve.index.year.unique():
        datos_anio = equity_curve[equity_curve.index.year == year]
        if len(datos_anio) > 0:
            valor_inicio = datos_anio.iloc[0]
            valor_fin = datos_anio.iloc[-1]
            retorno = (valor_fin - valor_inicio) / valor_inicio
            
            # Calcular métricas adicionales del año
            retornos_diarios = datos_anio.pct_change().dropna()
            volatilidad = retornos_diarios.std() * np.sqrt(252)
            
            # Max drawdown del año
            rolling_max = datos_anio.cummax()
            drawdown = (datos_anio - rolling_max) / rolling_max
            max_dd = drawdown.min()
            
            metricas_por_anio[year] = {
                'valor_inicio': valor_inicio,
                'valor_fin': valor_fin,
                'retorno': retorno,
                'volatilidad': volatilidad,
                'max_drawdown': max_dd,
                'dias_trading': len(datos_anio)
            }
    
    return metricas_por_anio

def generar_tabla_comparativa_anual(resultados_portafolio, resultados_benchmark, perfil_nombre):
    """Genera una tabla comparativa de métricas anuales."""
    
    metricas_port = calcular_metricas_anuales(resultados_portafolio['equity_curve'])
    metricas_bench = calcular_metricas_anuales(resultados_benchmark['equity_curve'])
    
    # Crear datos para la tabla
    datos = []
    for year in sorted(metricas_port.keys()):
        if year in metricas_bench:
            port = metricas_port[year]
            bench = metricas_bench[year]
            
            datos.append({
                'Año': year,
                f'Retorno {perfil_nombre}': f"{port['retorno']*100:.2f}%",
                'Retorno SPY': f"{bench['retorno']*100:.2f}%",
                'Alpha': f"{(port['retorno'] - bench['retorno'])*100:+.2f}%",
                f'Volatilidad {perfil_nombre}': f"{port['volatilidad']*100:.2f}%",
                'Volatilidad SPY': f"{bench['volatilidad']*100:.2f}%",
                f'Max DD {perfil_nombre}': f"{port['max_drawdown']*100:.2f}%",
                'Max DD SPY': f"{bench['max_drawdown']*100:.2f}%",
                'Días Trading': port['dias_trading']
            })
    
    return pd.DataFrame(datos)

def graficar_comparacion_anual(resultados_portafolio, resultados_benchmark, perfil_nombre):
    """Genera gráfico de barras comparativo por año."""
    
    metricas_port = calcular_metricas_anuales(resultados_portafolio['equity_curve'])
    metricas_bench = calcular_metricas_anuales(resultados_benchmark['equity_curve'])
    
    years = sorted(metricas_port.keys())
    retornos_port = [metricas_port[y]['retorno'] * 100 for y in years]
    retornos_bench = [metricas_bench[y]['retorno'] * 100 for y in years]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('📊 Retornos Anuales', '📉 Volatilidad Anual'),
        horizontal_spacing=0.15
    )
    
    # Gráfico de retornos
    fig.add_trace(
        go.Bar(name=f'Portafolio {perfil_nombre}', x=[str(y) for y in years], y=retornos_port,
               marker_color='#2E86AB', text=[f'{r:.1f}%' for r in retornos_port], textposition='outside'),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(name='SPY (Benchmark)', x=[str(y) for y in years], y=retornos_bench,
               marker_color='#E94F37', text=[f'{r:.1f}%' for r in retornos_bench], textposition='outside'),
        row=1, col=1
    )
    
    # Gráfico de volatilidad
    vol_port = [metricas_port[y]['volatilidad'] * 100 for y in years]
    vol_bench = [metricas_bench[y]['volatilidad'] * 100 for y in years]
    
    fig.add_trace(
        go.Bar(name=f'Vol {perfil_nombre}', x=[str(y) for y in years], y=vol_port,
               marker_color='#2E86AB', text=[f'{v:.1f}%' for v in vol_port], textposition='outside',
               showlegend=False),
        row=1, col=2
    )
    fig.add_trace(
        go.Bar(name='Vol SPY', x=[str(y) for y in years], y=vol_bench,
               marker_color='#E94F37', text=[f'{v:.1f}%' for v in vol_bench], textposition='outside',
               showlegend=False),
        row=1, col=2
    )
    
    fig.update_layout(
        title=f'📅 Comparación Anual: Portafolio {perfil_nombre} vs SPY',
        barmode='group',
        template='plotly_white',
        height=450,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
    )
    
    fig.update_yaxes(title_text="Retorno (%)", ticksuffix="%", row=1, col=1)
    fig.update_yaxes(title_text="Volatilidad (%)", ticksuffix="%", row=1, col=2)
    
    return fig

# ==========================================
# EJECUTAR COMPARACIÓN ANUAL
# ==========================================

if 'resultados_portafolio' in dir() and resultados_portafolio is not None:
    print("=" * 80)
    print("📅 COMPARACIÓN ANUAL: PORTAFOLIO vs BENCHMARK (SPY)")
    print("=" * 80)
    
    # Tabla de métricas anuales
    df_anual = generar_tabla_comparativa_anual(resultados_portafolio, resultados_benchmark, perfil)
    display(df_anual)
    
    # Análisis por año
    metricas_port = calcular_metricas_anuales(resultados_portafolio['equity_curve'])
    metricas_bench = calcular_metricas_anuales(resultados_benchmark['equity_curve'])
    
    print("\n" + "=" * 80)
    print("📊 ANÁLISIS DETALLADO POR AÑO")
    print("=" * 80)
    
    for year in sorted(metricas_port.keys()):
        port = metricas_port[year]
        bench = metricas_bench[year]
        alpha = port['retorno'] - bench['retorno']
        
        print(f"\n📆 AÑO {year}:")
        print(f"   • Portafolio: {port['retorno']*100:+.2f}% | SPY: {bench['retorno']*100:+.2f}%")
        print(f"   • Alpha generado: {alpha*100:+.2f}%")
        
        if alpha > 0:
            print(f"   ✅ El portafolio SUPERÓ al benchmark en {year}")
        else:
            print(f"   ❌ El portafolio tuvo underperformance en {year}")
    
    # Gráfico
    fig_anual = graficar_comparacion_anual(resultados_portafolio, resultados_benchmark, perfil)
    fig_anual.show()
    
else:
    print("⚠️ Ejecuta las celdas anteriores primero.")

📅 COMPARACIÓN ANUAL: PORTAFOLIO vs BENCHMARK (SPY)


,Año,Retorno Agresivo,Retorno SPY,Alpha,Volatilidad Agresivo,Volatilidad SPY,Max DD Agresivo,Max DD SPY,Días Trading
0,2024,42.89%,24.00%,+18.90%,24.75%,12.59%,-14.56%,-8.41%,252
1,2025,21.90%,16.34%,+5.56%,32.62%,19.76%,-30.93%,-19.00%,243



📊 ANÁLISIS DETALLADO POR AÑO

📆 AÑO 2024:
   • Portafolio: +42.89% | SPY: +24.00%
   • Alpha generado: +18.90%
   ✅ El portafolio SUPERÓ al benchmark en 2024

📆 AÑO 2025:
   • Portafolio: +21.90% | SPY: +16.34%
   • Alpha generado: +5.56%
   ✅ El portafolio SUPERÓ al benchmark en 2025


### 8.7 Exportar Métricas del Backtesting

In [33]:
# ============================================================================
# EXPORTAR MÉTRICAS DEL BACKTESTING
# ============================================================================

def exportar_resultados_backtest(resultados_portafolio, resultados_benchmark, perfil_nombre, output_dir='../reports'):
    """Exporta los resultados del backtesting a archivos CSV."""
    
    import os
    
    # Crear directorio si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 1. Métricas comparativas
    metricas_dict = {
        'Metrica': [
            'Capital_Inicial', 'Valor_Final', 'Retorno_Total', 'Retorno_Anualizado',
            'Volatilidad_Anualizada', 'Sharpe_Ratio', 'Sortino_Ratio', 
            'Max_Drawdown', 'Calmar_Ratio', 'Dias_Trading'
        ],
        f'Portafolio_{perfil_nombre}': [
            resultados_portafolio['capital_inicial'],
            resultados_portafolio['valor_final'],
            resultados_portafolio['retorno_total'],
            resultados_portafolio['retorno_anualizado'],
            resultados_portafolio['volatilidad_anualizada'],
            resultados_portafolio['sharpe_ratio'],
            resultados_portafolio['sortino_ratio'],
            resultados_portafolio['max_drawdown'],
            resultados_portafolio['calmar_ratio'],
            resultados_portafolio['dias_trading']
        ],
        'SPY_Benchmark': [
            resultados_benchmark['capital_inicial'],
            resultados_benchmark['valor_final'],
            resultados_benchmark['retorno_total'],
            resultados_benchmark['retorno_anualizado'],
            resultados_benchmark['volatilidad_anualizada'],
            resultados_benchmark['sharpe_ratio'],
            resultados_benchmark['sortino_ratio'],
            resultados_benchmark['max_drawdown'],
            resultados_benchmark['calmar_ratio'],
            resultados_benchmark['dias_trading']
        ]
    }
    df_metricas_export = pd.DataFrame(metricas_dict)
    metricas_path = f'{output_dir}/backtest_metricas_{perfil_nombre.lower()}.csv'
    df_metricas_export.to_csv(metricas_path, index=False)
    
    # 2. Equity curves
    equity_df = pd.DataFrame({
        'Fecha': resultados_portafolio['equity_curve'].index,
        f'Portafolio_{perfil_nombre}': resultados_portafolio['equity_curve'].values,
        'SPY_Benchmark': resultados_benchmark['equity_curve'].values
    })
    equity_path = f'{output_dir}/backtest_equity_curves_{perfil_nombre.lower()}.csv'
    equity_df.to_csv(equity_path, index=False)
    
    # 3. Retornos mensuales
    ret_mensual_port = calcular_metricas_mensuales(resultados_portafolio['equity_curve'])
    ret_mensual_bench = calcular_metricas_mensuales(resultados_benchmark['equity_curve'])
    
    mensual_df = pd.DataFrame({
        'Mes': ret_mensual_port.index,
        f'Retorno_Portafolio_{perfil_nombre}': ret_mensual_port.values,
        'Retorno_SPY': ret_mensual_bench.values
    })
    mensual_path = f'{output_dir}/backtest_retornos_mensuales_{perfil_nombre.lower()}.csv'
    mensual_df.to_csv(mensual_path, index=False)
    
    # 4. Composición del portafolio
    composicion_df = pd.DataFrame({
        'Ticker': list(resultados_portafolio['posiciones'].keys()),
        'Acciones': list(resultados_portafolio['posiciones'].values()),
        'Peso': [1/len(resultados_portafolio['posiciones'])] * len(resultados_portafolio['posiciones'])
    })
    composicion_path = f'{output_dir}/backtest_composicion_{perfil_nombre.lower()}.csv'
    composicion_df.to_csv(composicion_path, index=False)
    
    return {
        'metricas': metricas_path,
        'equity': equity_path,
        'mensual': mensual_path,
        'composicion': composicion_path
    }

# ==========================================
# EJECUTAR EXPORTACIÓN
# ==========================================

if 'resultados_portafolio' in dir() and resultados_portafolio is not None:
    print("=" * 70)
    print("💾 EXPORTANDO RESULTADOS DEL BACKTESTING")
    print("=" * 70)
    
    archivos = exportar_resultados_backtest(
        resultados_portafolio, 
        resultados_benchmark, 
        perfil
    )
    
    print("\n✅ Archivos exportados exitosamente:")
    for nombre, ruta in archivos.items():
        print(f"   📄 {nombre}: {ruta}")
    
    print("\n" + "=" * 70)
    print("📊 RESUMEN FINAL DEL BACKTESTING")
    print("=" * 70)
    print(f"\n🎯 Perfil de Inversión: {perfil}")
    print(f"📅 Período: {FECHA_INICIO.strftime('%Y-%m-%d')} a {FECHA_FIN.strftime('%Y-%m-%d')}")
    print(f"💰 Capital Inicial: ${CAPITAL_INICIAL:,.2f}")
    print(f"\n📈 Resultados del Portafolio:")
    print(f"   • Valor Final: ${resultados_portafolio['valor_final']:,.2f}")
    print(f"   • Retorno Total: {resultados_portafolio['retorno_total']*100:.2f}%")
    print(f"   • Sharpe Ratio: {resultados_portafolio['sharpe_ratio']:.3f}")
    print(f"\n📊 Resultados del Benchmark (SPY):")
    print(f"   • Valor Final: ${resultados_benchmark['valor_final']:,.2f}")
    print(f"   • Retorno Total: {resultados_benchmark['retorno_total']*100:.2f}%")
    print(f"   • Sharpe Ratio: {resultados_benchmark['sharpe_ratio']:.3f}")
    
    # Veredicto final
    alpha = resultados_portafolio['retorno_total'] - resultados_benchmark['retorno_total']
    print(f"\n🏆 ALPHA GENERADO: {alpha*100:+.2f}%")
    
else:
    print("⚠️ Ejecuta las celdas anteriores primero.")

💾 EXPORTANDO RESULTADOS DEL BACKTESTING

✅ Archivos exportados exitosamente:
   📄 metricas: ../reports/backtest_metricas_agresivo.csv
   📄 equity: ../reports/backtest_equity_curves_agresivo.csv
   📄 mensual: ../reports/backtest_retornos_mensuales_agresivo.csv
   📄 composicion: ../reports/backtest_composicion_agresivo.csv

📊 RESUMEN FINAL DEL BACKTESTING

🎯 Perfil de Inversión: Agresivo
📅 Período: 2024-01-02 a 2025-12-19
💰 Capital Inicial: $10,000.00

📈 Resultados del Portafolio:
   • Valor Final: $17,484.62
   • Retorno Total: 74.85%
   • Sharpe Ratio: 0.985

📊 Resultados del Benchmark (SPY):
   • Valor Final: $14,383.79
   • Retorno Total: 43.84%
   • Sharpe Ratio: 0.961

🏆 ALPHA GENERADO: +31.01%
